In [8]:
import pandas as pd
import numpy as np
from rectools.dataset import Interactions, Dataset, DenseFeatures
from rectools.model_selection import Splitter, TimeRangeSplitter
from rectools.models.base import ModelBase
from rectools.models import RandomModel, PopularModel
from rectools.metrics.base import MetricAtK
from rectools.metrics import (
    Precision,
    Recall,
    MAP
    NDCG,
    Serendipity,
    MeanInvUserFreq,
    IntraListDiversity,
    PairwiseHammingDistanceCalculator,
    calc_metrics,
    MRR,
    serendipity
)
from rectools import Columns
from typing import Dict, Union, Callable, Tuple, Any, List
from tqdm import tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')


In [2]:
interactions_df = pd.read_csv('interactions.csv')
users = pd.read_csv('users.csv')
items = pd.read_csv('items.csv')

In [3]:
metrics = {
    'precision@1': Precision(k=1),
    'precision@5': Precision(k=5),
    'precision@10': Precision(k=10),
    'recall@1': Recall(k=1),
    'recall@5': Recall(k=5),
    'recall@10': Recall(k=10),
    'MAP@1': MAP(k=1, divide_by_k=False),
    'MAP@5': MAP(k=5, divide_by_k=False),
    'MAP@10': MAP(k=10, divide_by_k=False),
    'NDCG@1': NDCG(k=1, log_base=2),
    'NDCG@5': NDCG(k=5, log_base=2),
    'NDCG@10': NDCG(k=10, log_base=2),
    'novelty@1': MeanInvUserFreq(k=1),
    'novelty@5': MeanInvUserFreq(k=5),
    'novelty@10': MeanInvUserFreq(k=10),
    "mrr@1": MRR(k=1),
    "mrr@5": MRR(k=5),
    "mrr@10": MRR(k=10)
}


In [4]:
def cross_validate(models, metrics, interactions, splitter, k_recos):
    columns = ['k_fold', 'model'] + [metric_name for metric_name in metrics]
    results_df = pd.DataFrame(columns=columns)

    fold_iterator = splitter.split(interactions, collect_fold_stats=True)

    for id_train, id_test, k_fold in fold_iterator:
        print(f"Split Index: {k_fold['i_split']}")
        print(f"Start Date: {k_fold['start']}")
        print(f"End Date: {k_fold['end']}")
        print(f"Train Set Size: {k_fold['train']}")
        print(f"Train Users: {k_fold['train_users']}")
        print(f"Train Items: {k_fold['train_items']}")
        print(f"Test Set Size: {k_fold['test']}")
        print(f"Test Users: {k_fold['test_users']}")
        print(f"Test Items: {k_fold['test_items']}")
        print("-" * 40) 

        train = interactions.df.iloc[id_train]
        dataset = Dataset.construct(train)
        val = interactions.df.iloc[id_test][Columns.UserItem]
        val_id = np.unique(val[Columns.User])

        catalog = train[Columns.Item].unique()
        
        for model_name, model in models.items():
            model = deepcopy(model)
            model.fit(dataset)
            recos = model.recommend(
                users=val_id,
                dataset=dataset,
                k=k_recos,
                filter_viewed=True,
            )
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=val,
                prev_interactions=train,
                catalog=catalog,
            )
            temp_df = pd.DataFrame([{"k_fold": k_fold["i_split"], "model": model_name, **metric_values}])
            results_df = pd.concat([results_df, temp_df], ignore_index=True)

    return results_df

In [9]:
interactions_df.rename(
    columns={"last_watch_dt": Columns.Datetime, "total_dur": rectools.Columns.Weight}, inplace=True
)
interactions = Interactions(interactions_df)

In [10]:
splitter = TimeRangeSplitter("9D", 3)
models = {"random": RandomModel(random_state=32), "popular": PopularModel()}
result = cross_validate(models, metrics, interactions, splitter, 10)

Split Index: 0
Start Date: 2021-07-27 00:00:00
End Date: 2021-08-05 00:00:00
Train Set Size: 3941089
Train Users: 749447
Train Items: 15109
Test Set Size: 314621
Test Users: 108975
Test Items: 7055
----------------------------------------
Split Index: 1
Start Date: 2021-08-05 00:00:00
End Date: 2021-08-14 00:00:00
Train Set Size: 4423600
Train Users: 819309
Train Items: 15331
Test Set Size: 335644
Test Users: 114204
Test Items: 7022
----------------------------------------
Split Index: 2
Start Date: 2021-08-14 00:00:00
End Date: 2021-08-23 00:00:00
Train Set Size: 4923625
Train Users: 888245
Train Items: 15554
Test Set Size: 379102
Test Users: 127438
Test Items: 7095
----------------------------------------


In [11]:
final_df = result.groupby('model').mean()
display(final_df)

,precision@1,precision@5,precision@10,recall@1,recall@5,recall@10,MAP@1,MAP@5,MAP@10,NDCG@1,NDCG@5,NDCG@10,novelty@1,novelty@5,novelty@10,mrr@1,mrr@5,mrr@10
model,,,,,,,,,,,,,,,,,,
popular,0.083015,0.057349,0.03703,0.043862,0.142082,0.178335,0.043862,0.081071,0.087131,0.083015,0.063278,0.047044,2.390990,3.073708,3.711110,0.083015,0.142431,0.149599
random,0.000190,0.000200,0.00020,0.000059,0.000310,0.000623,0.000059,0.000136,0.000177,0.000190,0.000196,0.000197,15.606846,15.597717,15.598941,0.000190,0.000444,0.000571


In [12]:
def visualize(model, interactions_data, users, k_recos, item_data):
    dataset = Dataset.construct(interactions_data)
    recos = model.recommend(
        users=users,
        dataset=dataset,
        k=k_recos,
        filter_viewed=True,
    )
    user_info = item_data[['item_id', 'title', 'genres']]
    user_info = user_info.assign(num_of_views=interactions_data.groupby('item_id')['user_id'].count())
    for user_id in users:
        us_info = interactions_data[interactions_data['user_id'] == user_id].merge(user_info, on="item_id")
        user_recos = recos[recos['user_id'] == user_id].merge(user_info, on="item_id")
        us_info = us_info[['user_id', 'item_id', 'title', 'genres', 'num_of_views']]
        print(f'User {user_id} watched:\n')
        display(us_info)
        print(f'\nRecommendations for user {user_id}:\n')
        display(user_recos)
        print("-" * 60)


In [13]:
model = PopularModel()
dataset = Dataset.construct(interactions_df)
model.fit(dataset)
user_ids = [666262, 672861, 955527]

In [15]:
visualize(model, interactions_df, users=user_ids, k_recos=10, item_data=items)

User 666262 watched:



,user_id,item_id,title,genres,num_of_views
0,666262,7957,Последний викинг,"боевики, историческое, приключения",1.0
1,666262,4785,Робин Гуд: Начало,"боевики, триллеры, приключения",1.0
2,666262,12981,Томирис,"боевики, драмы, историческое, военные",5.0



Recommendations for user 666262:



,user_id,item_id,score,rank,title,genres,num_of_views
0,666262,10440,202457.0,1,Хрустальный,"триллеры, детективы",4.0
1,666262,15297,193123.0,2,Клиника счастья,"драмы, мелодрамы",20.0
2,666262,9728,132865.0,3,Гнев человеческий,"боевики, триллеры",143.0
3,666262,13865,122119.0,4,Девятаев,"драмы, военные, приключения",821.0
4,666262,4151,91167.0,5,Секреты семейной жизни,комедии,10.0
5,666262,3734,74803.0,6,Прабабушка легкого поведения,комедии,NaN
6,666262,2657,68581.0,7,Подслушано,"драмы, триллеры",725.0
7,666262,4880,55043.0,8,Афера,комедии,5.0
8,666262,142,45367.0,9,Маша,"драмы, триллеры",15.0
9,666262,6809,40372.0,10,Дуров,документальное,14.0


------------------------------------------------------------
User 672861 watched:



...

------------------------------------------------------------
User 955527 watched:



,user_id,item_id,title,genres,num_of_views
0,955527,1183,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1.0
1,955527,13371,Пеле: Рождение легенды,"драмы, спорт, биография",2.0
2,955527,4725,Лобановский навсегда,"спорт, биография, документальное",7.0
3,955527,1238,Диего Марадона,"спорт, биография, документальное",642.0



Recommendations for user 955527:



,user_id,item_id,score,rank,title,genres,num_of_views
0,955527,10440,202457.0,1,Хрустальный,"триллеры, детективы",4.0
1,955527,15297,193123.0,2,Клиника счастья,"драмы, мелодрамы",20.0
2,955527,9728,132865.0,3,Гнев человеческий,"боевики, триллеры",143.0
3,955527,13865,122119.0,4,Девятаев,"драмы, военные, приключения",821.0
4,955527,4151,91167.0,5,Секреты семейной жизни,комедии,10.0
5,955527,3734,74803.0,6,Прабабушка легкого поведения,комедии,NaN
6,955527,2657,68581.0,7,Подслушано,"драмы, триллеры",725.0
7,955527,4880,55043.0,8,Афера,комедии,5.0
8,955527,142,45367.0,9,Маша,"драмы, триллеры",15.0
9,955527,6809,40372.0,10,Дуров,документальное,14.0


------------------------------------------------------------


In [16]:
model = RandomModel(random_state=32)
dataset = Dataset.construct(interactions_df)
model.fit(dataset)
user_ids = [666262, 672861, 955527]

In [17]:
visualize(model, interactions_df, users=user_ids, k_recos=10, item_data=items)

User 666262 watched:



,user_id,item_id,title,genres,num_of_views
0,666262,7957,Последний викинг,"боевики, историческое, приключения",1.0
1,666262,4785,Робин Гуд: Начало,"боевики, триллеры, приключения",1.0
2,666262,12981,Томирис,"боевики, драмы, историческое, военные",5.0



Recommendations for user 666262:



,user_id,item_id,score,rank,title,genres,num_of_views
0,666262,10101,10,1,Возвращение Будулая,мелодрамы,2.0
1,666262,619,9,2,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии",9.0
2,666262,12618,8,3,Пропавшая грамота,"фэнтези, комедии",7.0
3,666262,5967,7,4,Братья вне игры,"драмы, спорт",1.0
4,666262,4041,6,5,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики",368.0
5,666262,5701,5,6,Алые паруса: Новая история,"комедии, мелодрамы",5.0
6,666262,9738,4,7,Женщина в беде 3,"детективы, мелодрамы",54.0
7,666262,15247,3,8,Гордость и предубеждение,"драмы, мелодрамы",276.0
8,666262,10004,2,9,Болванчики,"мультфильм, приключения, комедии",14.0
9,666262,2816,1,10,Избави нас от лукавого,"ужасы, триллеры, детективы",72.0


------------------------------------------------------------
User 672861 watched:



...

------------------------------------------------------------
User 955527 watched:



,user_id,item_id,title,genres,num_of_views
0,955527,1183,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1.0
1,955527,13371,Пеле: Рождение легенды,"драмы, спорт, биография",2.0
2,955527,4725,Лобановский навсегда,"спорт, биография, документальное",7.0
3,955527,1238,Диего Марадона,"спорт, биография, документальное",642.0



Recommendations for user 955527:



,user_id,item_id,score,rank,title,genres,num_of_views
0,955527,496,10,1,Воскресший Эртугрул,"боевики, драмы, приключения",12.0
1,955527,4205,9,2,Дело гастронома №1 (Операция Беркут),"драмы, русские",3.0
2,955527,10822,8,3,Она защищает Родину,"драмы, советские, военные",2.0
3,955527,10914,7,4,Великолепная,"зарубежные, комедии, мелодрамы",136.0
4,955527,3999,6,5,Джиперс криперс,"ужасы, триллеры",1.0
5,955527,15756,5,6,Ремнант: Всё ещё вижу тебя (жестовым языком),"фантастика, зарубежные, триллеры",23.0
6,955527,14961,4,7,Битва за Землю,"боевики, ужасы, фантастика, триллеры",NaN
7,955527,13734,3,8,Сексуальный массаж и Фантазии,для взрослых,28.0
8,955527,3407,2,9,Черный капитан,"боевики, русские, военные",47.0
9,955527,14614,1,10,Настя,"мелодрамы, комедии",99.0


------------------------------------------------------------
